In [3]:
import pandas as pd
import numpy as np
from scipy import stats


data = {
    "ID": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
    "Age": [25, 37, np.nan, 29, 42, 55, 23, 29, np.nan, 45, np.nan],
    "Gender": ["Male", "Female", "Male", "Female", "Male", "Female", "Male", "Female", np.nan, "Female", "Male"],
    "Income": [50000, 62000, 45000, 54000, np.nan, 100000, 52000, 70000, np.nan, 85000, 200000],
    "Occupation": ["Engineer", "Data Scientist", "Product Manager", "Product Manager", "Sales", "Executive",
                   np.nan, "Product Manager", np.nan, "Executive", "Executive"],
    "Years_Employed": [2, 5, 3, np.nan, 8, 30, 1, 6, 15, np.nan, np.nan],
    "Satisfaction_Level": [0.8, 0.6, 0.7, 0.7, 0.5, 0.9, 0.4, 0.7, 0.9, np.nan, np.nan],
    "Purchase_History": ["High", "Medium", "Low", "High", "Low", "High", "Medium", "High", "Low", "High", "High"]
}

df = pd.DataFrame(data)

print("Initial Data:")
print(df)

Initial Data:
    ID   Age  Gender    Income       Occupation  Years_Employed  \
0    1  25.0    Male   50000.0         Engineer             2.0   
1    2  37.0  Female   62000.0   Data Scientist             5.0   
2    3   NaN    Male   45000.0  Product Manager             3.0   
3    4  29.0  Female   54000.0  Product Manager             NaN   
4    5  42.0    Male       NaN            Sales             8.0   
5    6  55.0  Female  100000.0        Executive            30.0   
6    7  23.0    Male   52000.0              NaN             1.0   
7    8  29.0  Female   70000.0  Product Manager             6.0   
8    9   NaN     NaN       NaN              NaN            15.0   
9   10  45.0  Female   85000.0        Executive             NaN   
10  11   NaN    Male  200000.0        Executive             NaN   

    Satisfaction_Level Purchase_History  
0                  0.8             High  
1                  0.6           Medium  
2                  0.7              Low  
3            

In [4]:
print("\nMissing values (isnull):")
print(df.isnull().sum())

print("\nNon-missing values (notnull):")
print(df.notnull().sum())


Missing values (isnull):
ID                    0
Age                   3
Gender                1
Income                2
Occupation            2
Years_Employed        3
Satisfaction_Level    2
Purchase_History      0
dtype: int64

Non-missing values (notnull):
ID                    11
Age                    8
Gender                10
Income                 9
Occupation             9
Years_Employed         8
Satisfaction_Level     9
Purchase_History      11
dtype: int64


In [11]:
# Strategy used: 
# - Fill missing Occupation with "Unknown"
# - Fill Satisfaction_Level with mean
# - Drop rows with too many missing values (e.g., row 9 and 11)

df["Occupation"].fillna("Unknown", inplace=True)
df["Satisfaction_Level"].fillna(df["Satisfaction_Level"].mean(), inplace=True)


df.dropna(subset=["Age", "Income", "Years_Employed"], how='all', inplace=True)


def satisfaction_to_label(x):
    return "High" if x > 0.7 else "Low"

df["Satisfaction_Label"] = df["Satisfaction_Level"].apply(satisfaction_to_label)


purchase_map = {"High": 2, "Medium": 1, "Low": 0}
df["Purchase_History_Score"] = df["Purchase_History"].map(purchase_map)

In [12]:
income_clean = df["Income"].dropna()
z_scores = stats.zscore(income_clean)


z_scores_series = pd.Series(z_scores, index=income_clean.index)

outliers_z = df.loc[z_scores_series[np.abs(z_scores_series) > 3].index]

print("\nOutliers using Z-Score:")
print(outliers_z[["ID", "Income"]])


Outliers using Z-Score:
Empty DataFrame
Columns: [ID, Income]
Index: []


In [13]:
Q1 = df["Income"].quantile(0.25)
Q3 = df["Income"].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers_iqr = df[(df["Income"] < lower_bound) | (df["Income"] > upper_bound)]
print("\nOutliers using IQR:")
print(outliers_iqr[["ID", "Income"]])


Outliers using IQR:
    ID    Income
10  11  200000.0


In [14]:
print("\nMissing Years_Employed before fillna:")
print(df["Years_Employed"].isnull().sum())


Missing Years_Employed before fillna:
0


In [15]:
median_years = df["Years_Employed"].median()
df["Years_Employed"].fillna(median_years, inplace=True)

In [16]:
print("\nMissing Years_Employed after fillna:")
print(df["Years_Employed"].isnull().sum())


Missing Years_Employed after fillna:
0


In [17]:
print("\nCleaned & Transformed Dataset:")
print(df)


Cleaned & Transformed Dataset:
    ID   Age  Gender    Income       Occupation  Years_Employed  \
0    1  25.0    Male   50000.0         Engineer             2.0   
1    2  37.0  Female   62000.0   Data Scientist             5.0   
2    3   NaN    Male   45000.0  Product Manager             3.0   
3    4  29.0  Female   54000.0  Product Manager             5.5   
4    5  42.0    Male       NaN            Sales             8.0   
5    6  55.0  Female  100000.0        Executive            30.0   
6    7  23.0    Male   52000.0          Unknown             1.0   
7    8  29.0  Female   70000.0  Product Manager             6.0   
8    9   NaN     NaN       NaN          Unknown            15.0   
9   10  45.0  Female   85000.0        Executive             5.5   
10  11   NaN    Male  200000.0        Executive             5.5   

    Satisfaction_Level Purchase_History Satisfaction_Label  \
0             0.800000             High               High   
1             0.600000           Medium